<a href="https://colab.research.google.com/github/wassimchouchen/Automatic-Speech-Recognition-/blob/main/Identify_medical_vocabulary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip install pydub
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding( 
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib
import pydub

from sklearn.cluster import AgglomerativeClustering
import numpy as np
import torch

# Check if a GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
num_speakers = 2 
language = 'any' 
model_size = 'medium'

In [3]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving conv.mp3 to conv.mp3


In [4]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'
# Load the audio file
audio_file = pydub.AudioSegment.from_wav(path)
audio_file = audio_file.set_channels(1)
audio_file.export(path, format="wav")
with wave.open(path) as f:
  print(f.getnchannels())

1


In [5]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 89.8MiB/s]


In [6]:
result = model.transcribe(path)
segments = result["segments"]

In [7]:
text=result["text"]

In [8]:
text

" podcastfrancaisfacile.com Bonjour. Bonjour, installez-vous, je vous en prie. Merci. Alors, qu'est-ce qui vous amène ? Eh bien, j'ai des douleurs au ventre, au niveau de l'estomac. Vous avez mal à l'estomac ? En fait, je ne suis pas sûr que ce soit à l'estomac. Vous avez mal en permanence ? Non, ce n'est pas tout le temps, c'est surtout après manger. Comment est la douleur ? Ce n'était pas très fort au début, mais depuis quelques jours, c'est très aigu. Quand ont commencé les premières douleurs ? C'était il y a deux mois environ. Je pensais que ça passerait, mais finalement, c'est de pire en pire. Bien, on va regarder ça. Enlevez votre chemise et allongez-vous."

In [9]:
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.3 MB/s eta 0:00:00


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [11]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [12]:
sequence_to_classify = text
candidate_labels = ['anatomie', 'pelvien', "périnée", "vagin", "vaginal", "anus", "rectal",
"rectum", "utérus", "utérin", "urètre", "cavité", "endocavitaire" , "examens", "touché vaginale", 
"touché mammaire", "touché abdominal", "touché rectale", "échographie", "frottis", "prélèvement" ,
"instruments médicaux","speculum", "sondes", "gants", "lubrifiant" , "pathologies" , "HPV", "IST", "MST", "incontinence",
"endométriose", "douleur", "périnéale"]
result=classifier(sequence_to_classify, candidate_labels, multi_class=True)
for i,j in zip(result["labels"],result["scores"]):
  print(i,":",j)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


IST : 0.9583739638328552
douleur : 0.9473059773445129
touché vaginale : 0.9448296427726746
prélèvement : 0.9176927804946899
touché rectale : 0.8872992396354675
périnée : 0.8730621933937073
sondes : 0.8632349967956543
examens : 0.8509401679039001
périnéale : 0.8501414656639099
instruments médicaux : 0.8457878232002258
touché abdominal : 0.8373198509216309
touché mammaire : 0.7941744923591614
endométriose : 0.7644859552383423
utérus : 0.7605158686637878
lubrifiant : 0.752054750919342
urètre : 0.7226285338401794
vagin : 0.7224775552749634
speculum : 0.7067820429801941
cavité : 0.6419246792793274
utérin : 0.6372186541557312
pathologies : 0.5768846273422241
pelvien : 0.5765195488929749
endocavitaire : 0.5741416215896606
échographie : 0.5716391801834106
frottis : 0.5353065133094788
MST : 0.452586829662323
gants : 0.43350064754486084
rectal : 0.42395782470703125
incontinence : 0.36439913511276245
rectum : 0.3372897505760193
vaginal : 0.3206106424331665
anatomie : 0.29866838455200195
HPV : 0.1